# Working with Xarray

[Xarray](http://xarray.pydata.org/) is an evolution of rasterio and is inspired by libraries like pandas to work with raster datasets. It also integrates tightly with [dask](https://dask.org/) that allows one to scale raster data processing using parallel computing.

[rioxarray](https://corteva.github.io/rioxarray/stable/index.html) is an extension of xarray that makes it easy to work with geospatial rasters. 

You can install the `xarray` and `rioxarray` packages from the `conda-forge` channel to use these libraries

In [3]:
import xarray as xr
import rioxarray as rxr

In [2]:
import os
data_pkg_path = 'data'
srtm_dir = 'srtm'
filename = 'N28E087.hgt'
path = os.path.join(data_pkg_path, srtm_dir, filename)

In [69]:
rds = xr.open_rasterio(path)

In [70]:
rds.attrs

{'transform': (0.0002777777777777778,
  0.0,
  85.99986111111112,
  0.0,
  -0.0002777777777777778,
  28.000138888888888),
 'crs': '+init=epsg:4326',
 'res': (0.0002777777777777778, 0.0002777777777777778),
 'is_tiled': 0,
 'nodatavals': (-32768.0,),
 'scales': (1.0,),
 'offsets': (0.0,),
 'units': ('m',)}

In [71]:
rds.values

array([[[3186, 3200, 3220, ..., 5338, 5334, 5332],
        [3171, 3178, 3200, ..., 5335, 5332, 5329],
        [3145, 3155, 3173, ..., 5332, 5330, 5327],
        ...,
        [ 368,  368,  366, ..., 1885, 1889, 1891],
        [ 364,  364,  362, ..., 1881, 1884, 1878],
        [ 360,  359,  357, ..., 1872, 1870, 1863]]], dtype=int16)

In [72]:
rds.shape

(1, 3601, 3601)

In [73]:
band1 = rds.squeeze()
band1.shape

(3601, 3601)

In [74]:
rds = rioxarray.open_rasterio(path, masked=True)

In [75]:
rds.rio.crs

CRS.from_epsg(4326)

# Merging Rasters

In [62]:
from rioxarray.merge import merge_arrays

In [53]:
srtm_path = os.path.join(data_pkg_path, 'srtm')
all_files = os.listdir(srtm_path)
print(all_files)

['N28E086.hgt', 'N28E087.hgt', 'N27E087.hgt', 'N27E086.hgt']


In [58]:
dataset_list = []
for file in all_files:
    path = os.path.join(srtm_path, file)
    dataset_list.append(rioxarray.open_rasterio(path, masked=True).squeeze())


In [63]:
merged = merge_arrays(dataset_list)

In [64]:
output_filename = 'merged.tif'
output_dir = 'output'
output_path = os.path.join(output_dir, output_filename)

In [66]:
merged.rio.to_raster(output_path, tiled=True)